# Now Building CKD model after spliting the data into training and test part 
### (We can split the data into training, test and validation part but the observation points is too small)

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).



### Load all the relavent libraries

In [0]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV,train_test_split
from sklearn.metrics import recall_score
from imblearn.over_sampling import SMOTE
from sklearn import metrics

In [0]:
#! pip install imblearn

### Load the Consolidated CKD Dataset for further processing

In [0]:
CKD = pd.read_csv("/content/consolidate_ckd.csv")

In [0]:
CKD.head()

,id,race,gender,age,creatinine_no_of_test,creatinine_mean_value,creatinine_duration_of_test,glucose_no_of_test,glucose_mean_value,glucose_duration_of_test,HGB_no_of_test,HGB_mean_value,HGB_duration_of_test,ldl_no_of_test,ldl_mean_value,ldl_duration_of_test,SBP_no_of_test,SBP_mean_value,SBP_duration_of_test,DBP_no_of_test,DBP_mean_value,DBP_duration_of_test,atenolol,atorvastatin,bisoprolol,canagliflozin,carvedilol,dapagliflozin,irbesartan,labetalol,losartan,lovastatin,metformin,metoprolol,nebivolol,olmesartan,pitavastatin,pravastatin,propranolol,rosuvastatin,simvastatin,telmisartan,valsartan,total_no_of_days_with_drug,total_no_of_days_without_drug,Stage_Progress
0,0,Unknown,Male,70,6,1.236667,2043,6,6.456667,2043,9,13.262222,5405,3,143.593333,489,8,139.610000,3169,8,83.173750,3169,NaN,180.0,NaN,NaN,NaN,NaN,NaN,NaN,450.0,NaN,660.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1290.0,328.0,True
1,1,White,Female,71,5,1.838000,1214,5,9.240000,1602,7,13.662857,4777,5,82.424000,1602,7,150.232857,2454,7,80.455714,2454,NaN,300.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,300.0,221.0,False
2,2,White,Female,57,5,2.170000,1737,3,6.466667,804,8,15.700000,4500,3,75.283333,681,4,140.210000,1242,4,89.217500,1242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,180.0,NaN,NaN,NaN,180.0,487.0,True
3,3,White,Male,74,5,0.996000,1800,6,6.525000,2475,10,13.198000,7395,5,98.246000,1670,7,139.588571,2963,7,74.857143,2963,NaN,360.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,360.0,301.0,False
4,4,White,Female,50,6,1.150000,2120,7,9.720000,1974,5,12.658000,2990,4,65.907500,1016,7,160.667143,2445,7,97.385714,2445,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,360.0,NaN,540.0,NaN,NaN,NaN,NaN,720.0,NaN,NaN,NaN,NaN,NaN,1620.0,481.0,True


In [0]:
CKD.Stage_Progress.value_counts()

False    200
True     100
Name: Stage_Progress, dtype: int64

In [0]:
CKD = CKD[[col for col in CKD.columns if col!='id']] ## removing id columns as it is unique val to identify
CKD.shape


(300, 45)

In [0]:
CKD_dummies = pd.get_dummies(data=CKD, columns=['race','gender'],drop_first=True)
CKD_dummies.shape

(300, 48)

In [0]:
CKD_dummies.head()

,age,creatinine_no_of_test,creatinine_mean_value,creatinine_duration_of_test,glucose_no_of_test,glucose_mean_value,glucose_duration_of_test,HGB_no_of_test,HGB_mean_value,HGB_duration_of_test,ldl_no_of_test,ldl_mean_value,ldl_duration_of_test,SBP_no_of_test,SBP_mean_value,SBP_duration_of_test,DBP_no_of_test,DBP_mean_value,DBP_duration_of_test,atenolol,atorvastatin,bisoprolol,canagliflozin,carvedilol,dapagliflozin,irbesartan,labetalol,losartan,lovastatin,metformin,metoprolol,nebivolol,olmesartan,pitavastatin,pravastatin,propranolol,rosuvastatin,simvastatin,telmisartan,valsartan,total_no_of_days_with_drug,total_no_of_days_without_drug,Stage_Progress,race_Black,race_Hispanic,race_Unknown,race_White,gender_Male
0,70,6,1.236667,2043,6,6.456667,2043,9,13.262222,5405,3,143.593333,489,8,139.610000,3169,8,83.173750,3169,NaN,180.0,NaN,NaN,NaN,NaN,NaN,NaN,450.0,NaN,660.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1290.0,328.0,True,0,0,1,0,1
1,71,5,1.838000,1214,5,9.240000,1602,7,13.662857,4777,5,82.424000,1602,7,150.232857,2454,7,80.455714,2454,NaN,300.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,300.0,221.0,False,0,0,0,1,0
2,57,5,2.170000,1737,3,6.466667,804,8,15.700000,4500,3,75.283333,681,4,140.210000,1242,4,89.217500,1242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,180.0,NaN,NaN,NaN,180.0,487.0,True,0,0,0,1,0
3,74,5,0.996000,1800,6,6.525000,2475,10,13.198000,7395,5,98.246000,1670,7,139.588571,2963,7,74.857143,2963,NaN,360.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,360.0,301.0,False,0,0,0,1,1
4,50,6,1.150000,2120,7,9.720000,1974,5,12.658000,2990,4,65.907500,1016,7,160.667143,2445,7,97.385714,2445,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,360.0,NaN,540.0,NaN,NaN,NaN,NaN,720.0,NaN,NaN,NaN,NaN,NaN,1620.0,481.0,True,0,0,0,1,0


In [0]:
CKD_dummies.isnull().sum()

age                                0
creatinine_no_of_test              0
creatinine_mean_value              0
creatinine_duration_of_test        0
glucose_no_of_test                 0
glucose_mean_value                 0
glucose_duration_of_test           0
HGB_no_of_test                     0
HGB_mean_value                     0
HGB_duration_of_test               0
ldl_no_of_test                     0
ldl_mean_value                     0
ldl_duration_of_test               0
SBP_no_of_test                     0
SBP_mean_value                     0
SBP_duration_of_test               0
DBP_no_of_test                     0
DBP_mean_value                     0
DBP_duration_of_test               0
atenolol                         274
atorvastatin                     191
bisoprolol                       295
canagliflozin                    296
carvedilol                       283
dapagliflozin                    298
irbesartan                       294
labetalol                        299
l

In [0]:
CKD_dummies.fillna(0,inplace=True)

In [0]:
CKD_dummies.isnull().sum()

age                              0
creatinine_no_of_test            0
creatinine_mean_value            0
creatinine_duration_of_test      0
glucose_no_of_test               0
glucose_mean_value               0
glucose_duration_of_test         0
HGB_no_of_test                   0
HGB_mean_value                   0
HGB_duration_of_test             0
ldl_no_of_test                   0
ldl_mean_value                   0
ldl_duration_of_test             0
SBP_no_of_test                   0
SBP_mean_value                   0
SBP_duration_of_test             0
DBP_no_of_test                   0
DBP_mean_value                   0
DBP_duration_of_test             0
atenolol                         0
atorvastatin                     0
bisoprolol                       0
canagliflozin                    0
carvedilol                       0
dapagliflozin                    0
irbesartan                       0
labetalol                        0
losartan                         0
lovastatin          

In [0]:
x = CKD_dummies[[col for col in CKD_dummies.columns if col!='Stage_Progress']]
y = CKD_dummies['Stage_Progress']
print(x.shape)
print(y.shape)

(300, 47)
(300,)


In [0]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.10,random_state=20)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(270, 47)
(30, 47)
(270,)
(30,)


In [0]:
model = RandomForestClassifier()
model.fit(x_train,y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [0]:
print(model.feature_importances_)

[0.04713056 0.01581723 0.08163246 0.04119506 0.01857724 0.05038758
 0.0391861  0.03155928 0.06642724 0.0477308  0.01902588 0.03765149
 0.04017072 0.02103606 0.04949616 0.04885292 0.0174916  0.0403701
 0.04258351 0.00578686 0.01675883 0.00096528 0.00045839 0.00368121
 0.00085493 0.00255452 0.00011107 0.01141877 0.00509641 0.02083384
 0.01621847 0.00265652 0.0008605  0.00013587 0.01561918 0.00070087
 0.00556758 0.01077743 0.00277709 0.00689081 0.03389604 0.04219523
 0.00283871 0.00190245 0.00376826 0.00542606 0.02292684]


In [0]:
y_pred = model.predict(x_test)

In [0]:
print('recall:-',metrics.recall_score(y_test,y_pred)*100)
print('precision:-',metrics.precision_score(y_test,y_pred)*100)
print(metrics.accuracy_score(y_test,y_pred)*100)
metrics.confusion_matrix(y_test,y_pred)

recall:- 20.0
precision:- 22.22222222222222
50.0


array([[13,  7],
       [ 8,  2]])

In [0]:
x_train.shape, y_train.shape

((270, 47), (270,))

In [0]:
##lets use SMOTE and Oversample minority classes
sm = SMOTE(random_state=20,ratio=1.0)
x_train, y_train = sm.fit_sample(x_train,y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [0]:
print(x_train.shape,y_train.shape)

(360, 47) (360,)


In [0]:
## lets apply random forest again and check the recall and precision
model_oversampling = RandomForestClassifier()
model_oversampling.fit(x_train,y_train)
y_pred_oversampling = model_oversampling.predict(x_test)
print('recall:-',metrics.recall_score(y_test,y_pred_oversampling)*100)
print('precision:-',metrics.precision_score(y_test,y_pred_oversampling)*100)
print('Accuracy:-',metrics.accuracy_score(y_test,y_pred_oversampling)*100)
metrics.confusion_matrix(y_test,y_pred_oversampling)

recall:- 20.0
precision:- 18.181818181818183
Accuracy:- 43.333333333333336


array([[11,  9],
       [ 8,  2]])

In [0]:
## lets try undersampling 
from imblearn.under_sampling import RandomUnderSampler
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.10,random_state=20)
underSample = RandomUnderSampler(sampling_strategy="majority")
x_train_undersampling, y_train_undersampling = underSample.fit_resample(x_train,y_train)
print(x_train_undersampling.shape, y_train_undersampling.shape)
model_undersampling = RandomForestClassifier()
model_undersampling.fit(x_train_undersampling,y_train_undersampling)
y_pred_undersampling = model_undersampling.predict(x_test)
print('recall:-',metrics.recall_score(y_test,y_pred_undersampling)*100)
print('precision:-',metrics.precision_score(y_test,y_pred_undersampling)*100)
print('Accuracy:-',metrics.accuracy_score(y_test,y_pred_undersampling)*100)
metrics.confusion_matrix(y_test,y_pred_undersampling)

(180, 47) (180,)
recall:- 30.0
precision:- 21.428571428571427
Accuracy:- 40.0


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


array([[ 9, 11],
       [ 7,  3]])

In [0]:
## as we see above our recall has doubled.. so if we are satisfied with increase in recall, because we woule like to report if 
## a person has progressed with more accuuracy.. that means we need more recalll score..TP/(TP_FN)

In [0]:
## lets use Grid search to optimize or tune RandomForest classifier parameteres.
from sklearn.model_selection import GridSearchCV,train_test_split
param_grid = {
    "n_estimators" : list(range(100,1000,100)),
    "min_samples_leaf" : list(range(1,10))
}
model_grid = RandomForestClassifier(random_state=10)
grid = GridSearchCV(estimator=model_grid, param_grid=param_grid)
grid.fit(x_train_undersampling,y_train_undersampling)
print(grid.best_score_)
print(grid.best_params_)

0.6444444444444445
{'min_samples_leaf': 8, 'n_estimators': 100}


100

In [0]:
## grid search has not helped much recall is still 40%
grid_model_with_best_params = RandomForestClassifier(n_estimators=grid.best_params_['n_estimators'],min_samples_leaf=grid.best_params_['min_samples_leaf']) 
grid_model_with_best_params.fit(x_train_undersampling,y_train_undersampling)
y_pred_with_best_params = grid_model_with_best_params.predict(x_test)
print('recall:-',metrics.recall_score(y_test,y_pred_with_best_params)*100)
print('precision:-',metrics.precision_score(y_test,y_pred_with_best_params)*100)
print('Accuracy:-',metrics.accuracy_score(y_test,y_pred_with_best_params)*100)
metrics.confusion_matrix(y_test,y_pred_with_best_params)

recall:- 40.0
precision:- 28.57142857142857
Accuracy:- 46.666666666666664


array([[10, 10],
       [ 6,  4]])